# 데이터 포함관계 확인
---
* 기본적인 EDA를 진행할 예정입니다.
* 데이터 사이의 포함관계 확인을 위해 작성했습니다.

## 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

## 데이터 로드

In [2]:
path = './data/'

In [3]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
history = pd.read_csv(path + 'historical_transactions.csv')
new_history = pd.read_csv(path + 'new_merchant_transactions.csv')
merchant = pd.read_csv(path + 'merchants.csv')

## 각 데이터 고유 인덱스 개수

* train 과 history, new_history의 외래키: card_id
* history, new_history와 merchant의 외래키: merchant_id

### card_id

In [4]:
print("train card_id 주어진 데이터: ",train.card_id.shape[0])
print("train card_id 고유 인덱스: ",len(train.card_id.unique()))

train card_id 주어진 데이터:  201917
train card_id 고유 인덱스:  201917


In [5]:
print("test card_id 주어진 데이터: ",test.card_id.shape[0])
print("test card_id 고유 인덱스: ",len(test.card_id.unique()))

test card_id 주어진 데이터:  123623
test card_id 고유 인덱스:  123623


In [6]:
print("history card_id 주어진 데이터: ", history.card_id.shape[0])
print("history card_id 고유 인덱스: ", len(history.card_id.unique()))

history card_id 주어진 데이터:  29112361
history card_id 고유 인덱스:  325540


In [7]:
print("new_history card_id 주어진 데이터: ", new_history.card_id.shape[0])
print("new_history card_id 고유 인덱스: ", len(new_history.card_id.unique()))

new_history card_id 주어진 데이터:  1963031
new_history card_id 고유 인덱스:  290001


In [8]:
# new_history가 history의 부분집합인가
set(new_history.card_id.unique()).issubset(set(history.card_id.unique()))

True

In [9]:
# train이 history의 부분집합인가
set(train.card_id.unique()).issubset(set(history.card_id.unique()))

True

In [10]:
# test가 history의 부분집합인가
set(test.card_id.unique()).issubset(set(history.card_id.unique()))

True

In [11]:
# train 과 test의 중복 확인
len(list(train.card_id.unique()) + list(test.card_id.unique())) - len(set(list(train.card_id.unique()) + list(test.card_id.unique())))

0

In [12]:
# train과 new history 중복 확인
len(list(new_history.card_id.unique()) + list(train.card_id.unique())) - len(set(list(new_history.card_id.unique()) + list(train.card_id.unique())))

179986

In [13]:
# test와 new history 중복 확인
len(list(new_history.card_id.unique()) + list(test.card_id.unique())) - len(set(list(new_history.card_id.unique()) + list(test.card_id.unique())))

110015

In [14]:
# history와 new history 사이에 중복된 행이 존재할까?
check = pd.concat([history, new_history], axis = 0)
print("history와 new history 합친 것 인덱스: ", check.shape[0])
print("history와 new history 중복제거 인덱스:", check.drop_duplicates().shape[0])

history와 new history 합친 것 인덱스:  31075392
history와 new history 중복제거 인덱스: 31075392


#### 결론
* history의 고유 인덱스의 개수 = train의 고유 인덱스의 개수 + test의 고유 인덱스의 개수
* new_history의 고유 인덱스는 history의 고유 인덱스의 부분집합 (new_history가 history의 부분집합이라는 소리가 아닙니다 !!)
* new_history와 history 사이에 중복된 행은 없음
* train의 고유 인덱스는 총 201917개, new_history와 train의 중복 인덱스는 총 179986개
* test의 고유 인덱스는 총 123623개, new_history와 test의 중복인덱스는 총 110015개
* 즉, train의 89%, test의 89%가 new_history와 관계있음

In [15]:
110015 / 123623

0.8899233961317877

### merchant_id

In [16]:
print("history merchant_id 주어진 데이터: ", history.merchant_id.shape[0])
print("history merchant_id 고유 인덱스: ", len(history.merchant_id.unique()))

history merchant_id 주어진 데이터:  29112361
history merchant_id 고유 인덱스:  326312


In [17]:
print("new_history merchant_id 주어진 데이터: ", new_history.merchant_id.shape[0])
print("new_history merchant_id 고유 인덱스: ", len(new_history.merchant_id.unique()))

new_history merchant_id 주어진 데이터:  1963031
new_history merchant_id 고유 인덱스:  226130


In [18]:
print("merchant merchant_id 주어진 데이터: ", merchant.merchant_id.shape[0])
print("merchant merchant_id 고유 인덱스: ", len(merchant.merchant_id.unique()))

merchant merchant_id 주어진 데이터:  334696
merchant merchant_id 고유 인덱스:  334633


In [19]:
# history와 new history의 교집합 개수
len(set(history.merchant_id.unique()).intersection(set(new_history.merchant_id.unique())))

217808

In [20]:
# history가 merchant의 부분집합인가
set(new_history.merchant_id.unique()).issubset(set(history.merchant_id.unique()))

False

In [21]:
# new_history가 merchant의 부분집합인가
set(new_history.merchant_id.unique()).issubset(set(merchant.merchant_id.unique()))

False

In [22]:
# history 부분집합이 아니다. 그럼 무엇때문에?
set(history.merchant_id.unique()).difference(set(merchant.merchant_id.unique()))

{nan}

In [23]:
# new history 부분집합이 아니다. 그럼 무엇때문에?
set(new_history.merchant_id.unique()).difference(set(merchant.merchant_id.unique()))

{nan}

In [24]:
# 널을 제거하고 부분집합인지 확인 - history와 merchant
set(history.merchant_id.dropna().unique()).issubset(set(merchant.merchant_id.unique()))

True

In [26]:
# 널을 제거하고 부분집합인지 확인 - new_history와 merchant
set(new_history.merchant_id.dropna().unique()).issubset(set(merchant.merchant_id.unique()))

True

#### 결론
* history의 merchant_id와 new_history의 merchant_id가 중복되는 경우가 많다. 케글 홈페이지의 new_merchant 데이터 설명이 ..?
* history의 merchant_id의 고유 인덱스는 merchant의 merchant_id의 고유 인덱스의 부분 집합 (단, null 제거 시)
* new_history의 merchant_id의 고유 인덱스는 merchant의 merchant_id의 고유 인덱스의 부분 집합 (단, null 제거 시)